In [1]:
! pip install datasets transformers
! pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 6)

pipe = pipeline("text-classification", model="bert-base-uncased", tokenizer = "bert-base-uncased")

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["lyrics"], max_length=40, padding="max_length", truncation=True)
    tokenized_inputs["labels"] = [labels_dict[label] for label in examples["genre"]]
    return tokenized_inputs

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
songs = pd.read_csv("combined_songs.csv")
train_data, test_data = train_test_split(songs, test_size = 0.2, random_state = 69)
songs_split = np.array_split(train_data, 8)

labels_dict = {'rock':0, 'country':1, 'r&b':2, 'rap & hip hop':3, 'pop':4, 'rap':5}

def data_loading(songs):
    train_data_dict = { 'genre': [str(ele["genre"]).lower() if not pd.isnull(ele["genre"]) else '' for _, ele in songs.iterrows()]
                        , 'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in songs.iterrows()]
                      }
    train_data = Dataset.from_dict(train_data_dict)
    # test_data_dict = {'genre': [str(ele["genre"]).lower() if not pd.isnull(ele["genre"]) else '' for _, ele in test_data.iterrows()]
    #                   , 'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in test_data.iterrows()]
    #                   }
    # test_data = Dataset.from_dict(test_data_dict)
    lm_train_data = train_data.map(tokenize_function, batched=True, num_proc=4)#, remove_columns=train_data.column_names)
    train_valid_data = lm_train_data.train_test_split(test_size=0.2)
    lm_train_data = train_valid_data["train"]
    lm_valid_data = train_valid_data["test"]
    # lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4)#, remove_columns=test_data.column_names)

    lm_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
    # lm_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
    return lm_train_data, lm_valid_data

In [10]:
test_data_dict = {'genre': [str(ele["genre"]).lower() if not pd.isnull(ele["genre"]) else '' for _, ele in test_data.iterrows()]
                  , 'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in test_data.iterrows()]
                  }
test_data = Dataset.from_dict(test_data_dict)
lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4)#, remove_columns=test_data.column_names)
lm_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/16857 [00:00<?, ? examples/s]

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted'),
    }
for song in songs_split:
    try:
      model = AutoModelForSequenceClassification.from_pretrained("task2", num_labels = 6)
    except:
      model = model

    lm_train_data, lm_valid_data = data_loading(song)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=lm_train_data,
        eval_dataset=lm_test_data,
        compute_metrics=compute_metrics,
    )
    trainer.train()

    trainer.save_model("task2")

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8429 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.239300,1.010070,0.565047,0.562039,0.560953,0.565047
2,0.837000,1.046850,0.575132,0.577316,0.588406,0.575132
3,0.525100,1.178067,0.573115,0.574667,0.579821,0.573115


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8429 [00:00<?, ? examples/s]

...

Map (num_proc=4):   0%|          | 0/8428 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.938100,0.887302,0.629175,0.629812,0.632257,0.629175
2,0.593500,0.996379,0.626090,0.627511,0.631007,0.626090
3,0.334100,1.301827,0.620632,0.622076,0.625618,0.620632


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8428 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.941800,0.892944,0.625793,0.629773,0.638939,0.625793
2,0.616400,0.988275,0.635285,0.636272,0.639595,0.635285
3,0.365200,1.289870,0.631370,0.631445,0.632568,0.631370


In [16]:
eval_results = trainer.evaluate(lm_test_data)
print(f"Accuracy: {eval_results['eval_accuracy']:.2f}")
print(f"F1: {eval_results['eval_f1']:.2f}")
print(f"Precision: {eval_results['eval_precision']:.2f}")
print(f"Recall: {eval_results['eval_recall']:.2f}")

Accuracy: 0.64
F1: 0.64
Precision: 0.64
Recall: 0.64


In [17]:
input_lyric = """
When I get on the dance floor with this girl who is a supermodel, I'm like, "Whoa!" It's like, you know, this isn't what she wants to do and she has to do this. And you can't be a good dancer.

You know, there are two things that I'm going out to do. One is to do a video on Instagram. It's not about the video. It's about the video. So what I've done is I have a video and a video and an image. So you can be a good dancer and be a good dancer but you'll have to learn how to do it. So that's why I'm in there doing this and I'm doing this.

So that's where the video comes into play.

Yeah, I'm a big, strong girl who's a great dancer. And I don't want you to be a good dancer and you won't be a good dancer. I'll tell ya what, I don't want to be a bad dancer. I'll tell ya what, I want to be great because I'm a big dancer. I'm a girl who wants to be great. But I'm a big, strong girl. I'm just a
"""


In [18]:
result = pipe(input_lyric)
print("Predicted Genre:", result[0]['label'])
print("Score:", result[0]['score'])

Predicted Genre: LABEL_0
Score: 0.622121274471283


In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
model.push_to_hub("guna-2222/NLP_task2")
tokenizer.push_to_hub("guna-2222/NLP_task2")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/guna-2222/NLP_task2/commit/0fed51e3723459c54d4b2faf9b2c86758ac7fed3', commit_message='Upload tokenizer', commit_description='', oid='0fed51e3723459c54d4b2faf9b2c86758ac7fed3', pr_url=None, pr_revision=None, pr_num=None)